## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-21-08-51-57 +0000,bbc,Households face unexpected rise in energy pric...,https://www.bbc.com/news/articles/c2lplg7d9ewo...
1,2025-11-21-08-51-00 +0000,wsj,Gilt Yields Fall After Weak U.K. Retail Sales ...,https://www.wsj.com/articles/u-s-treasurys-see...
2,2025-11-21-08-49-00 +0000,wsj,"Europe Must Look Inward to Drive Growth, ECB’s...",https://www.wsj.com/articles/europe-must-look-...
3,2025-11-21-08-48-00 +0000,wsj,Gold Poised for Weekly Loss Amid Uncertainty O...,https://www.wsj.com/finance/commodities-future...
4,2025-11-21-08-47-00 +0000,wsj,"Euro, European Yields Little Moved After PMI Data",https://www.wsj.com/finance/currencies/yuan-is...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2394/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
299,trump,43
6,new,21
295,will,14
121,ukraine,13
165,california,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
176,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...,109
41,2025-11-21-05-35-00 +0000,latimes,Race for California governor continues to heat...,https://www.latimes.com/california/story/2025-...,83
223,2025-11-20-20-12-26 +0000,latimes,Trump administration plans to open Pacific Coa...,https://www.latimes.com/environment/story/2025...,82
295,2025-11-20-16-22-10 +0000,bbc,Trump to meet incoming New York Mayor Zohran M...,https://www.bbc.com/news/articles/cvgmgddyjl0o...,81
170,2025-11-20-22-26-31 +0000,bbc,Trump signs bill ordering US justice departmen...,https://www.bbc.com/news/articles/cly2y40lj5go...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
176,109,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...
85,63,2025-11-21-01-36-00 +0000,wsj,The Transportation Department said it would pa...,https://www.wsj.com/politics/policy/air-traffi...
277,60,2025-11-20-17-20-47 +0000,nypost,Kamala Harris and Jill Biden come face-to-face...,https://nypost.com/2025/11/20/us-news/kamala-h...
278,53,2025-11-20-17-20-17 +0000,nypost,Rubio casts doubt on pro-Russian Ukraine peac...,https://nypost.com/2025/11/20/world-news/rubio...
300,52,2025-11-20-15-45-00 +0000,wsj,Home sales rose to an eight-month high in Octo...,https://www.wsj.com/economy/housing/home-sales...
156,42,2025-11-20-22-37-38 +0000,latimes,Driver charged with murder in crash that kille...,https://www.latimes.com/california/story/2025-...
320,37,2025-11-20-14-18-22 +0000,nyt,Israeli Strikes Kill at Least 25 in Gaza Amid ...,https://www.nytimes.com/2025/11/19/world/middl...
195,37,2025-11-20-21-33-00 +0000,wsj,The Energy Department is eliminating two of it...,https://www.wsj.com/politics/policy/energy-dep...
117,36,2025-11-21-00-04-21 +0000,latimes,L.A. Olympic organizing committee adds former ...,https://www.latimes.com/sports/story/2025-11-2...
284,35,2025-11-20-17-00-00 +0000,wsj,​Young voters are pessimistic about the nation...,https://www.wsj.com/politics/elections/nikki-h...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
